# Assembling the TOP dataset

### Import libraries

In [ ]:
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt
import glob
import sys
sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config

## Read data into pandas dataframe

How do we define which files should be used together?
options:
- all files in folder
- based on suffix (e.g. "n=895_06-Feb-2023_PVC2.tsv")
- check first columns to see whether it matches

For now, we will will use the first option, approved by Mathijs on 26th June 2023

In [ ]:
experiment_folder= 'TOP'

In [ ]:
config = Config()
root_directory = config.get_directory('raw_data')
if os.path.isdir(os.path.join(root_directory, experiment_folder)):
    print("this folder exists, we will take tsv from here")
else: 
    print("this folder does not seem to exist, try typing again")

In [ ]:
root_directory = config.get_directory('raw_data')
tsv_path = os.path.join(root_directory, experiment_folder)

tsv_files = [os.path.join(tsv_path, file) for file in os.listdir(tsv_path) if file.endswith('.tsv')]
tsv_files

# Add a step to get rid of second visits #TODO

In [ ]:
# Read files into dataframes
dataframes = [pd.read_csv(file, sep='\t', header=[0]) for file in tsv_files]
sample_df = dataframes[2]
cols = sample_df.columns
sample_df

In [ ]:
numbr = 0
for frame in dataframes:
    filepath = '../open_work/internal_results/stitchy' 
    filename = os.path.join(filepath,str(numbr+1)) 
    if not os.path.exists(filepath):
    # if filder doesn't exist, create it
        os.makedirs(filepath)
    frame.to_csv((filename +'.tsv'), sep="\t")
    numbr +=1

## Copy identical columns from any file

we could also read it from the data, but if it's always the same, we can just define it here

In [ ]:
tsv_folder_made = '../open_work/internal_results/stitchy'
identical_columns = sep.check_identical_columns(tsv_folder_made)

In [ ]:
identical_columns

In [ ]:
sample_df.columns

In [ ]:
#identical_columns

In [ ]:
stitched = sample_df[identical_columns].copy()

In [ ]:
n_identical = stitched.shape[1]

In [ ]:
#stitched['renumber'] = stitched.index

In [ ]:
#stitched.shape

In [ ]:
#Here we need to add back in patient ID

### Here is where we add the different parts to stitched

In [ ]:
for df in dataframes:
    for col in df.columns[n_identical:]:
        stitched[col] = df[col]

stitched.columns

## Here we can should get rid of second visits, but what we see is that session was not in the common columns. We will get rid of all second time points, and people ending in _2. And mention to scientists

In [ ]:
#stitched

In [ ]:
#stitched = stitched.loc[stitched[('LongitudinalTimePoint',           'integer')] == 'TimePoint_1']

In [ ]:
stitched

In [ ]:
stitched.columns

In [ ]:
stitched = stitched[1:]
#stitched

In [ ]:
stitched['session'].unique()

In [ ]:
#stitched.columns

In [ ]:
stitched['LongitudinalTimePoint'].unique()

## So in this dataset we have one longitudinal timepoint, and one type of session. no need to filter down away from this

In [ ]:
stitched.columns

In [ ]:
sexage_path = os.path.join(root_directory, 'age_data', 'Age_sex_TOP.csv')
sexage_df = pd.read_csv(sexage_path, index_col=0)
sexage_df['renumber'] = sexage_df.index
sexage_df['renumber'] = sexage_df['renumber'].apply(str)

sexage_df

## Here we take the patient ID and align it with our other frame's index

In [ ]:
stitched = stitched.reset_index(drop=False)
stitched

In [ ]:
#stitched = sep.concat_double_header(stitched)

In [ ]:
stitched.renumber

In [ ]:
#print(type(stitched['renumber'][9]))

In [ ]:
stitched['renumber'] = stitched['renumber'].astype('string')

In [ ]:
stitched['index']

In [ ]:
sexage_df = sexage_df.reset_index(drop=True)
sexage_df

In [ ]:
stitched.columns

In [ ]:
result = stitched.merge(sexage_df, on="renumber")
result

In [ ]:
result

In [ ]:
result

In [ ]:
result.columns

In [ ]:
result = result.loc[:,~result.columns.duplicated()].copy()

## Conform file to new standard

### example of new standard (from M.D. on 23/08/2023)

In [ ]:
standard_folder = 'standard' 

In [ ]:
standard_path = os.path.join(root_directory, standard_folder,'participants_CBA.csv')
#standard_path_file = standard_path

In [ ]:
standard = pd.read_csv(standard_path)
standard

In [ ]:
result.columns = result.columns.str.lower()
result

## Save off file

In [ ]:
filepath = '../open_work/internal_results' 
filename = os.path.join(filepath,'top_stitched_conformed.csv') 
if not os.path.exists(filepath):
    # if filder doesn't exist, create it
    os.makedirs(filepath)
result.to_csv(filename)  